# Plots Correlation vs Bin Size - might be a tad unreliable tho

In [1]:
import pandas as pd
import scipy.stats
from bokeh.plotting import figure , ColumnDataSource
from bokeh.io import show
import numpy as np
from bokeh.models import HoverTool

#______________________________________________________________________________________________________________________________

def find_nearest(array, value):

    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

#______________________________________________________________________________________________________________________________


# ______________________________________________________________________________________________________________________________


df1 = pd.read_csv('etch_roof_d3s.csv')
df2 = pd.read_csv('etch_roof_weather.csv')

#______________________________________________________________________________________________________________________________

df1 = df1[['deviceTime_unix', 'cpm']].copy()
df2 = df2[['deviceTime_unix', 'temperature']].copy()

unix_timerad = df1.as_matrix(columns=df1.columns[0:1]).ravel()
#print unix_timerad
unix_timeweather = df2.as_matrix(columns=df2.columns[0:1]).ravel()
#print unix_timeweather

if(unix_timerad[0]<unix_timeweather[0]):
    df2 = df2.drop(df2.index[0:find_nearest(unix_timeweather,unix_timerad[0])])
else:
    df1 = df1.drop(df1.index[0:find_nearest(unix_timerad, unix_timeweather[0])])


#______________________________________________________________________________________________________________________________
# Deal with the NaNs

df1 = df1.set_index(['deviceTime_unix'])
df1.index = pd.to_datetime(df1.index, unit='s')

df2 = df2.set_index(['deviceTime_unix'])
df2.index = pd.to_datetime(df2.index, unit='s')

corr = []
count = []


i = 1000

while i <= 80000:


    df_binned = df1.resample(str(i)+"T", label='right').mean().reset_index()
    df2_binned = df2.resample(str(i)+"T", label='right').mean().reset_index()
    df_binned = df_binned.drop(df_binned.index[len(df2_binned):])

    #print df_binned
    #print df2_binned


    indsr = pd.isnull(df_binned).any(1).nonzero()[0].ravel()
    indsw = pd.isnull(df2_binned).any(1).nonzero()[0].ravel()

    indsnan = np.concatenate([indsr,indsw])
    indsnan = np.unique(indsnan)

    j = len(indsnan)-1

    while (j>=0):
        df_binned = df_binned.drop(df_binned.index[indsnan[j]])
        df2_binned = df2_binned.drop(df2_binned.index[indsnan[j]])

        j = j-1

    #print df_binned
    #print df2_binned

    count.append(i)
    corr.append((scipy.stats.pearsonr(df_binned.as_matrix(columns=df_binned.columns[1:]).ravel(), df2_binned.as_matrix(columns=df2_binned.columns[1:]).ravel()))[0])

    i = i+2000


#______________________________________________________________________________________________________________________________
k = 1000
imgs = []
while k<=80000:
    imgs.append('file:/Users/albertqiang/PycharmProjects/DoseNet/images/'+str(k)+'plot.png') # This can be changed to the filepaths of the user's pngs
    k = k+2000
#______________________________________________________________________________________________________________________________

source = ColumnDataSource(
        data=dict(
            x=count,
            y=corr,
            #imgs = [
             #   'http://bokeh.pydata.org/static/snake.jpg',
              #  'http://bokeh.pydata.org/static/snake2.png',
               # 'http://bokeh.pydata.org/static/snake3D.png',
                #'http://bokeh.pydata.org/static/snake4_TheRevenge.png',
                #'http://bokeh.pydata.org/static/snakebite.jpg',
                #'file:///Users/albertqiang/Desktop/download.jpeg'
            #]
            imgs = imgs
        )
    )
hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs" height="500" alt="@imgs" width="500"
                    style="float: center; margin: 0px 0px 0px 0px;"
                    border="0"
                ></img>
            </div>
            
            
        </div>
        """
    )

p = figure(plot_width=1000, plot_height=1000, tools=[hover],
           )

p.line('x', 'y', source=source)

show(p)







        deviceTime_unix          cpm
0   2017-03-16 09:20:00  2144.163910
1   2017-03-17 02:00:00          NaN
2   2017-03-17 18:40:00          NaN
3   2017-03-18 11:20:00  2159.609709
4   2017-03-19 04:00:00          NaN
5   2017-03-19 20:40:00          NaN
6   2017-03-20 13:20:00  2174.314286
7   2017-03-21 06:00:00  2176.437000
8   2017-03-21 22:40:00  2189.008333
9   2017-03-22 15:20:00          NaN
10  2017-03-23 08:00:00          NaN
11  2017-03-24 00:40:00  2169.147739
12  2017-03-24 17:20:00  2170.934673
13  2017-03-25 10:00:00  2172.871591
14  2017-03-26 02:40:00          NaN
15  2017-03-26 19:20:00  2170.426667
16  2017-03-27 12:00:00  2175.075000
17  2017-03-28 04:40:00          NaN
18  2017-03-28 21:20:00          NaN
19  2017-03-29 14:00:00  2129.205634
20  2017-03-30 06:40:00  2125.806000
21  2017-03-30 23:20:00  2127.612500
22  2017-03-31 16:00:00          NaN
23  2017-04-01 08:40:00          NaN
24  2017-04-02 01:20:00          NaN
25  2017-04-02 18:00:00          NaN
2